# OPTIMIZACIÓN DEL MODELO

En el capítulo anterior construimos nuestro primer modelo de aprendizaje supervisado. Ahora queremos mejorar su precisión y reducir los efectos del sobreajuste. Un buen lugar para empezar es modificando los hiperparámetros del modelo.

Sin cambiar ningún otro hiperparámetro, primero vamos a modificar max_depth de "30" a "5". El modelo ahora genera los siguientes resultados:
```python
# Los resultados diferirán debido a la división aleatoria de los datos
Error Absoluto Medio del Conjunto de Entrenamiento: 129412.51
```
Aunque el error absoluto medio del conjunto de entrenamiento es más alto, esto ayuda a reducir el problema del sobreajuste y debería mejorar los resultados de los datos de prueba. Otro paso para optimizar el modelo es agregar más árboles. Si establecemos n_estimators en 250, obtenemos este resultado:
```python
# Los resultados diferirán debido a la división aleatoria de los datos
Error Absoluto Medio del Conjunto de Entrenamiento: 118130.46
Error Absoluto Medio del Conjunto de Prueba: 159886.32
```
Esta segunda optimización reduce la tasa de error absoluto del conjunto de entrenamiento en aproximadamente $11,000 y ahora tenemos una brecha más pequeña entre nuestros resultados de entrenamiento y prueba para el error absoluto medio.

Juntas, estas dos optimizaciones subrayan la importancia de maximizar y comprender el impacto de cada hiperparámetro individual. Si decides replicar este modelo de aprendizaje supervisado en casa, te recomiendo que pruebes modificando cada uno de los hiperparámetros individualmente y analices su impacto en el error absoluto medio. Además, notarás cambios en el tiempo de procesamiento de la máquina según los hiperparámetros seleccionados. Por ejemplo, establecer max_depth en "5" reduce el tiempo de procesamiento total en comparación con cuando estaba establecido en "30" porque el número máximo de capas de ramas es significativamente menor. La velocidad de procesamiento y los recursos se convertirán en una consideración importante a medida que avances en el trabajo con conjuntos de datos más grandes.

Otra técnica importante de optimización es la selección de características. Como recordarás, eliminamos nueve características mientras limpiábamos nuestro conjunto de datos. Ahora podría ser un buen momento para reconsiderar esas características y analizar si tienen un efecto en la precisión general del modelo. "SellerG" sería una característica interesante para agregar al modelo porque la empresa inmobiliaria que vende la propiedad podría tener algún impacto en el precio de venta final.

Alternativamente, eliminar características del modelo actual puede reducir el tiempo de procesamiento sin tener un efecto significativo en la precisión, o incluso puede mejorarla. Para seleccionar características de manera efectiva, es mejor aislar las modificaciones de características y analizar los resultados, en lugar de aplicar varios cambios a la vez.

Si bien el ensayo y error manual puede ser una técnica efectiva para comprender el impacto de la selección de variables e hiperparámetros, también hay técnicas automatizadas para la optimización del modelo, como la búsqueda en cuadrícula. La búsqueda en cuadrícula te permite enumerar un rango de configuraciones que deseas probar para cada hiperparámetro, y luego prueba metódicamente cada uno de esos posibles hiperparámetros. Se lleva a cabo un proceso de votación automatizado para determinar el modelo óptimo. ¡Como el modelo debe probar cada combinación posible de hiperparámetros, la búsqueda en cuadrícula tarda mucho tiempo en ejecutarse! Un ejemplo de código para la búsqueda en cuadrícula se muestra al final de este capítulo.

Por último, si deseas utilizar un algoritmo de aprendizaje supervisado diferente y no gradient boosting, gran parte del código utilizado en este ejercicio se puede replicar. Por ejemplo, el mismo código se puede usar para importar un nuevo conjunto de datos, previsualizar el dataframe, eliminar características (columnas), eliminar filas, dividir y mezclar el conjunto de datos, y evaluar el error absoluto medio.

http://scikit-learn.org es un gran recurso para aprender más sobre otros algoritmos, así como el gradient boosting utilizado en este ejercicio.

Para obtener una copia del código, por favor contacta al autor en oliver.theobald@scatterplotpress.com o consulta el ejemplo de código a continuación. Además, si tienes problemas para implementar el modelo utilizando el código encontrado en este libro, no dudes en contactar al autor por correo electrónico para obtener asistencia adicional sin costo alguno.

#### Code for the Optimized Model
```py
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_splitfrom sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib

# Read in data from CSV
df = pd.read_csv('~/Downloads/Melbourne_housing_FULL-26-09-2017.csv')

# Delete unneeded columns
del df['Address']
del df['Method']
del df['SellerG']
del df['Date']
del df['Postcode']
del df['Lattitude']
del df['Longtitude']
del df['Regionname']
del df['Propertycount']

# Remove rows with missing values
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

# Convert non-numerical data using one-hot encoding
features_df = pd.get_dummies(df, columns=['Suburb', 'CouncilArea', 'Type'])

# Remove price
del features_df['Price']

# Create X and y arrays from the dataset
X = features_df.as_matrix()
y = df['Price'].as_matrix()

# Split data into test/train set (70/30 split) and shuffle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Set up algorithm
model = ensemble.GradientBoostingRegressor(
    n_estimators=250,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=4,
    min_samples_leaf=6,
    max_features=0.6,
    loss='huber'
)

# Run model on training data
model.fit(X_train, y_train)# Save model to file
joblib.dump(model, 'trained_model.pkl')

# Check model accuracy (up to two decimal places)
mse = mean_absolute_error(y_train, model.predict(X_train))
print ("Training Set Mean Absolute Error: %.2f" % mse)
mse = mean_absolute_error(y_test, model.predict(X_test))
print ("Test Set Mean Absolute Error: %.2f" % mse)
```

#### Code for Grid Search Model

```py
# Import libraries, including GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

# Read in data from CSV
df = pd.read_csv('~/Downloads/Melbourne_housing_FULL-26-09-2017.csv')

# Delete unneeded columns
del df['Address']
del df['Method']
del df['SellerG']
del df['Date']
del df['Postcode']
del df['Lattitude']
del df['Longtitude']
del df['Regionname']
del df['Propertycount']

# Remove rows with missing values
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

# Convert non-numerical data using one-hot encoding
features_df = pd.get_dummies(df, columns=['Suburb', 'CouncilArea', 'Type'])

# Remove price
del features_df['Price']

# Create X and y arrays from the dataset
X = features_df.as_matrix()
y = df['Price'].as_matrix()

# Split data into test/train set (70/30 split) and shuffleX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# Input algorithm
model = ensemble.GradientBoostingRegressor()

# Set the configurations that you wish to test
param_grid = {
    'n_estimators': [300, 600, 1000],
    'max_depth': [7, 9, 11],
    'min_samples_split': [3, 4, 5],
    'min_samples_leaf': [5, 6, 7],
    'learning_rate': [0.01, 0.02, 0.6, 0.7],
    'max_features': [0.8, 0.9],
    'loss': ['ls', 'lad', 'huber']
}

# Define grid search. Run with four CPUs in parallel if applicable.
gs_cv = GridSearchCV(model, param_grid, n_jobs=4)

# Run grid search on training data
gs_cv.fit(X_train, y_train)

# Print optimal hyperparameters
print(gs_cv.best_params_)

# Check model accuracy (up to two decimal places)
mse = mean_absolute_error(y_train, gs_cv.predict(X_train))
print("Training Set Mean Absolute Error: %.2f" % mse)
mse = mean_absolute_error(y_test, gs_cv.predict(X_test))
print("Test Set Mean Absolute Error: %.2f" % mse)

# FURTHER RESOURCES

This section lists relevant learning materials for readers that wish to progress further in the field of machine learning. Please note that certain details listed in this section, including prices, may be subject to change in the future.

### | Machine Learning |
[Machine Learning](https://www.coursera.org/learn/machine-learning)
**Format:** Coursera course
**Presenter:** Andrew Ng
**Cost:** Free
**Suggested Audience:** Beginners (especially those with a preference for MATLAB)

A free and well-taught introduction from Andrew Ng, one of the most influential figures in this field. This course has become a virtual rite of passage for anyone interested in machine learning.

[Project 3: Reinforcement Learning](https://inst.eecs.berkeley.edu/~cs188/sp12/projects/reinforcement/reinforcement.html)
**Format:** Online blog tutorial
**Author:** EECS Berkeley
**Suggested Audience:** Upper intermediate to advanced
A practical demonstration of reinforcement learning, and Q-learning specifically, explained through the game Pac-Man.

### | Basic Algorithms |

[Machine Learning With Random Forests And Decision Trees: A Visual Guide For Beginners](https://www.amazon.com/Machine-Learning-Random-Forests-Decision-ebook/dp/B01JBL8YVK/ref=sr_1_1?s=digital-text&ie=UTF8&qid=1517105184&sr=1-1&keywords=Machine+Learning+With+Random+Forests+And+Decision+Trees%3A+A+Visual+Guide+For+Beginners)
**Format:** E-book
**Author:** Scott Hartshorn
**Suggested Audience:** Established beginnersA short, affordable (USD $3.20), and engaging read on decision trees and random forests with detailed visual examples, useful practical tips, and clear instructions.

**Linear Regression And Correlation: A Beginner's Guide**
**Format:** E-book
**Author:** Scott Hartshorn
**Suggested Audience:** All
A well-explained and affordable (USD $3.20) introduction to linear regression, as well as correlation.

### | The Future of AI |

[The Inevitable: Understanding the 12 Technological Forces That Will Shape Our Future](https://www.amazon.com/Inevitable-Understanding-Technological-Forces-Future-ebook/dp/B016JPTOUG/ref=sr_1_1?s=digital-text&ie=UTF8&qid=1517104544&sr=1-1&keywords=The+Inevitable%3A+Understanding+the+12+Technological+Forces+That+Will+Shape+Our+Future)
**Format:** E-Book, Book, Audiobook
**Author:** Kevin Kelly
**Suggested Audience:** All (with an interest in the future)
A well-researched look into the future with a major focus on AI and machine learning by The New York Times Best Seller Kevin Kelly. Provides a guide to twelve technological imperatives that will shape the next thirty years.

[Homo Deus: A Brief History of Tomorrow](https://www.amazon.com/Homo-Deus-Brief-History-Tomorrow-ebook/dp/B019CGXTP0/ref=sr_1_1?ie=UTF8&qid=1517399025&sr=8-1&keywords=Homo+Deus%3A+A+Brief+History+of+Tomorrow)
**Format:** E-Book, Book, Audiobook
**Author:** Yuval Noah Harari
**Suggested Audience:** All (with an interest in the future)
As a follow-up title to the success of Sapiens: A Brief History of Mankind, Yuval Noah Harari examines the possibilities of the future with notable sections of the book examining machine consciousness, applications in AI, and the immense power of data and algorithms.

### | Programming |

[Learning Python, 5th EditionFormat: ](https://www.amazon.com/Learning-Python-5th-Mark-Lutz/dp/1449355730/)
**Format:** E-Book, Book
**Author:** Mark Lutz
**Suggested Audience:** All (with an interest in learning Python)
A comprehensive introduction to Python published by O’Reilly Media.

[Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow-ebook/dp/B06XNKV5TS/ref=sr_1_4?s=digital-text&ie=UTF8&qid=1517104226&sr=1-4&keywords=machine+learning)
**Format:** E-Book, Book
**Author:** Aurélien Géron
**Suggested Audience:** All (with an interest in programming in Python, Scikit-Learn and TensorFlow)
As a highly popular O’Reilly Media book written by machine learning consultant Aurélien Géron, this is an excellent advanced resource for anyone with a solid foundation of machine learning and computer programming.

### | Recommendation Systems |
[The Netflix Prize and Production Machine Learning Systems: An Insider Look](http://blogs.mathworks.com/loren/2015/04/22/the-netflix-prize-and-production-machine-learning-systems-an-insider-look/)
**Format:** Blog
**Author:** Mathworks
**Suggested Audience:** All
A very interesting blog article demonstrating how Netflix applies machine learning to form movie recommendations.

[Recommender Systems](https://www.coursera.org/specializations/recommender-systems)
**Format:** Coursera course
**Presenter:** The University of Minnesota
**Cost:** Free 7-day trial or included with $49 USD Coursera subscription
**Suggested Audience:** All
Taught by the University of Minnesota, this Coursera specialization covers fundamental recommender system techniques including content-based and collaborative filtering as well as non-personalized and project-association recommender systems.

### | Deep Learning |
[Deep Learning Simplified](https://www.youtube.com/watch?v=b99UVkWzYTQ)
**Format:** Blog
**Channel:** DeepLearning.TV
**Suggested Audience:** All
A short video series to get you up to speed with deep learning. Available for free on YouTube.

[Deep Learning Specialization: Master Deep Learning, and Break into AI](https://www.coursera.org/specializations/deep-learning)
**Format:** Coursera course
**Presenter:** deeplearning.ai and NVIDIA
**Cost:** Free 7-day trial or included with $49 USD Coursera subscription
**Suggested Audience:** Intermediate to advanced (with experience in Python)
A robust curriculum for those wishing to learn how to build neural networks in Python and TensorFlow, as well as career advice, and how deep learning theory applies to industry.

[Deep Learning Nanodegree](https://www.udacity.com/course/deep-learning-nanodegree-foundation--nd101)
**Format:** Udacity course
**Presenter:** Udacity
**Cost:** $599 USD
**Suggested Audience:** Upper beginner to advanced, with basic experience in Python
Comprehensive and practical introduction to convolutional neural networks, recurrent neural networks, and deep reinforcement learning taught online over a four-month period. Practical components include building a dog breed classifier, generating TV scripts, generating faces, and teaching a quadcopter how to fly.

### | Future Careers |
[Will a Robot Take My Job?](http://www.bbc.com/news/technology-34066941)
**Format:** Online articleAuthor: The BBC
**Suggested Audience:** All
Check how safe your job is in the AI era leading up to the year 2035.

[So You Wanna Be a Data Scientist? A Guide to 2015's Hottest Profession](http://mashable.com/2014/12/25/data-scientist/)
**Format:** Blog
**Author:** Todd Wasserman
**Suggested Audience:** All
Excellent insight into becoming a data scientist.

[The Data Science Venn Diagram](http://drewconway.com/zia/2013/3/26/the-data-science-venn-diagram)
**Format:** Blog
**Author:** Drew Conway
**Suggested Audience:** All
The popular 2010 data science diagram designed by Drew Conway.

# DESCARGA DE CONJUNTOS DE DATOS

Antes de que puedas comenzar a practicar algoritmos y construir modelos de aprendizaje automático, primero necesitarás datos. Para los principiantes que comienzan en el aprendizaje automático, hay varias opciones. Una es obtener tu propio conjunto de datos escribiendo un rastreador web en Python o utilizando una herramienta de arrastrar y soltar como Import.io para explorar Internet. Sin embargo, la opción más fácil y mejor para comenzar es visitar kaggle.com.

Como se menciona a lo largo de este libro, Kaggle ofrece conjuntos de datos gratuitos para descargar. Esto te ahorra el tiempo y el esfuerzo de obtener y formatear tu propio conjunto de datos. Mientras tanto, también tienes la oportunidad de discutir y resolver problemas con otros usuarios en el foro, unirte a competiciones y simplemente charlar sobre datos.

Sin embargo, ten en cuenta que los conjuntos de datos que descargues de Kaggle necesitarán inherentemente cierto refinamiento (a través de la limpieza) para adaptarse al modelo de aprendizaje automático que decidas construir. A continuación, se presentan cuatro conjuntos de datos de muestra gratuitos de Kaggle que pueden ser útiles para tu aprendizaje en este campo.

#### [Informe Mundial de Felicidad](mailto:https://www.kaggle.com/unsdsn/world-happiness)

¿Qué países ocupan los primeros lugares en felicidad general? ¿Qué factores contribuyen más a la felicidad? ¿Cómo cambiaron los rankings de los países entre los informes de 2015 y 2016? ¿Experimentó algún país un aumento o disminución significativa en la felicidad? Estas son las preguntas que puedes hacer con este conjunto de datos que registra puntajes y rankings de felicidad utilizando datos de la Encuesta Mundial de Gallup. Los puntajes se basan en respuestas a las principales preguntas de evaluación de la vida realizadas en la encuesta.

#### [Reseñas de Hoteles](https://www.kaggle.com/datafiniti/hotel-reviews)

¿Tener una reputación de cinco estrellas conduce a más huéspedes insatisfechos, y, por el contrario, pueden los hoteles de dos estrellas mejorar las calificaciones de los huéspedes al establecer expectativas bajas y superarlas? ¿O los hoteles con una y dos estrellas simplemente tienen bajas calificaciones por una razón? Descubre todo esto a partir de este conjunto de datos de muestra de reseñas de hoteles. Este conjunto de datos en particular cubre 1,000 hoteles e incluye nombre del hotel, ubicación, fecha de la reseña, texto, título, nombre de usuario y calificación. El conjunto de datos se obtiene de la Base de Datos Empresariales de Datafiniti, que incluye casi todos los hoteles del mundo.

#### [Conjunto de Datos de Cervezas Artesanales](https://www.kaggle.com/nickhould/craft-cans)

¿Te gusta la cerveza artesanal? Este conjunto de datos contiene una lista de 2,410 cervezas artesanales estadounidenses y 510 cervecerías recopiladas en enero de 2017 de CraftCans.com. Beber y analizar datos es perfectamente legal.

#### [Reembolsos de la Cámara de Diputados de Brasil](https://www.kaggle.com/epattaro/brazils-house-of-deputies-reimbursements)

Como los políticos en Brasil tienen derecho a recibir reembolsos por el dinero gastado en actividades para "servir mejor al pueblo", hay hallazgos interesantes y valores atípicos sospechosos que se pueden encontrar en este conjunto de datos. Los datos sobre estos gastos están disponibles públicamente, pero hay muy poco monitoreo de gastos en Brasil. Así que no te sorprendas al ver a un servidor público acumulando más de 800 vuelos en doce meses, y otro que registró R 140,000 (USD $44,500) en gastos de correo postal, ¡sí, correo postal!